In [1]:
# Import Libraries
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio import Affine
from rasterio.enums import Resampling
from rasterstats import point_query
from geojson import Feature, Point, FeatureCollection
import earthpy.plot as ep
from scipy import ndimage, stats
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
# Directories and Source Files
work_dir = '/Users/arbailey/natcap/idb/data/work/sentinel'
tile = 'T16QDF'

# All are in UTM16N, WGS-84, except for TNC raster
# SDB bathy rasters for the blue/red and blue/green ratios
sdb_red_file =  tile + "_sdb_red.tif" 
sdb_grn_file = tile + "_sdb_grn.tif" 
sdb_red_source = os.path.join(work_dir, sdb_red_file)
sdb_grn_source = os.path.join(work_dir, sdb_grn_file)

# TNC raster  
tnc_source = '/Users/arbailey/natcap/idb/data/source/tnc/bathy_mar/tnc_bathy_MAR_32616.vrt'

# CONABIO bathy (4m grid) as points
conabio_path = '/Users/arbailey/natcap/idb/data/source/conabio/batimv2gw_c/batimv2uw_pts.csv'

# Belize Lidar (2m grid) as points
bzlidar_path = '/Users/arbailey/natcap/idb/data/source/bathy/bzczmai/bzczmai_bathy_pts.csv'

In [3]:
# Import Sentinel-2 bathy and TNC bathy rasters with rasterio
print(sdb_red_source)
with rasterio.open(sdb_red_source) as src:
    sdb_red_profile = src.profile
    sdb_red_np = src.read(1)
    print(sdb_red_np)
    print(sdb_red_profile)
    
print(sdb_grn_source)
with rasterio.open(sdb_grn_source) as src:
    sdb_grn_profile = src.profile
    sdb_grn_np = src.read(1)
    print(sdb_grn_np)
    print(sdb_grn_profile)
    
print(tnc_source)
with rasterio.open(tnc_source) as src:
    tnc_profile = src.profile
    tnc_np = src.read(1)
    print(tnc_np)
    print(tnc_profile)

/Users/arbailey/natcap/idb/data/work/sentinel/T16QDF_sdb_red.tif
[[-9.9990000e+03 -9.9990000e+03 -9.9990000e+03 ...  5.9397697e-01
   5.4462147e-01 -9.4703770e-01]
 [-9.9990000e+03 -9.9990000e+03 -9.9990000e+03 ...  1.5838814e-01
   1.0010452e+00  5.9087467e-01]
 [-9.9990000e+03 -9.9990000e+03 -1.5556593e+00 ...  6.6680145e-01
   1.9336014e+00  1.9338369e+00]
 ...
 [-9.9990000e+03 -9.9990000e+03 -9.9990000e+03 ...  2.5529385e-01
   3.2267475e-01  9.8939419e-01]
 [-9.9990000e+03 -9.9990000e+03 -9.9990000e+03 ...  9.0121460e-01
   1.7607555e+00  2.1694021e+00]
 [-9.9990000e+03 -9.9990000e+03 -9.9990000e+03 ...  9.6288681e-01
   5.6234550e-01  9.4233799e-01]]
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -9999.0, 'width': 10980, 'height': 10980, 'count': 1, 'crs': CRS.from_epsg(32616), 'transform': Affine(10.0, 0.0, 399960.0,
       0.0, -10.0, 2100000.0), 'tiled': False, 'interleave': 'band'}
/Users/arbailey/natcap/idb/data/work/sentinel/T16QDF_sdb_grn.tif
[[-9.9990000e+03 -9.9990000

In [4]:
# Mask out the raster no data values
sdb_red_masked = np.ma.masked_values(sdb_red_np, sdb_red_profile['nodata'])
sdb_grn_masked = np.ma.masked_values(sdb_grn_np, sdb_grn_profile['nodata'])
tnc_masked = np.ma.masked_values(tnc_np, tnc_profile['nodata'])
sdb_grn_masked

masked_array(
  data=[[--, --, --, ..., 11.145851135253906, 10.091606140136719,
         -0.5768661499023438],
        [--, --, --, ..., 5.8847808837890625, 8.013648986816406,
         6.3569183349609375],
        [--, --, -8.93206787109375, ..., 13.035545349121094,
         13.541778564453125, 16.033126831054688],
        ...,
        [--, --, --, ..., 8.488075256347656, 10.0548095703125,
         19.466651916503906],
        [--, --, --, ..., 9.868797302246094, 12.284317016601562,
         17.167770385742188],
        [--, --, --, ..., 9.065986633300781, 9.922225952148438,
         13.025764465332031]],
  mask=[[ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        [ True,  True, False, ..., False, False, False],
        ...,
        [ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False]],
  fill_value=-9999.0,
  dtype=f

In [ ]:
conabio_df = pd.read_csv(conabio_path)
conabio_gdf = gpd.GeoDataFrame(conabio_df, geometry=gpd.points_from_xy(conabio_df['X'], conabio_df['Y']))
conabio_gdf.head()

In [ ]:
bzlidar_df = pd.read_csv(bzlidar_path)
bzlidar_gdf = gpd.GeoDataFrame(bzlidar_df, geometry=gpd.points_from_xy(bzlidar_df['X'], bzlidar_df['Y']))
bzlidar_gdf.head()

In [ ]:
# # SDB raster bathy values sampled from raster at CONABIO depth point locations  (uses rasterstats)
# conabio_pts_grn = point_query(conabio_gdf, sdb_grn_masked, 
#                            affine=relbathy_profile['transform'], 
#                            geojson_out=True,
#                            property_name='sdb_grn',
#                            nodata=-99999
#                           )
# conabio_pts_grn  # GeoJSOn object   -- list of GeoJSON objects
# collection = FeatureCollection(relbathy_pts)
# conabio_pts_grn_gdf = gpd.GeoDataFrame.from_features(collection['features'])

In [ ]:
# conabio_pts_grn_red = point_query(conabio_pts_grn, sdb_red_masked, 
#                            affine=relbathy_profile['transform'], 
#                            geojson_out=True,
#                            property_name='sdb_red',
#                            nodata=-99999
#                           )
# conabio_pts_grn_red  # GeoJSOn object   -- list of GeoJSON objects
